# Table of Contents
*     Dataset Information
*         Data description
*         Files
*     Preliminary Data Exploration
*         Install and Import Libraries
*         Load Data
*         General Dataset Information
*     Exploratory Data Analysis
*         Text Length
*         Word Count
*         Distribution of top n-grams
*         Distribution of top n-grams for each discourse type
*         Text Distribution
*         Word Clouds
*         Displaying Labels
*         NER: Name Entity Recognition
*         Text Clustering
*         Word Embeddings Visualization


# Data Description


The dataset contains argumentative essays written by U.S students in grades 6-12. The essays were annotated by expert raters for elements commonly found in argumentative writing.
The task is to predict the human annotations. You will first need to segment each essay into discrete rhetorical and argumentative elements (i.e., discourse elements) and then classify each element as one of the following: Lead - an introduction that begins with a statistic, a quotation, a description, or some other device to grab the reader’s attention and point toward the thesis

***Position***- an opinion or conclusion on the main question

***Claim*** - a claim that supports the position

***Counterclaim*** - a claim that refutes another claim or gives an opposing reason to the position

***Rebuttal***- a claim that refutes a counterclaim

***Evidence***- ideas or examples that support claims, counterclaims, or rebuttals.

***Concluding Statement***- a concluding statement that restates the claims


# Files
***train.zip*** - folder of individual .txt files, with each file containing the full text of an essay response in the training set

***train.csv*** - file containing the annotated version of all essays in the training set

***test.zip*** - folder of individual .txt files, with each file containing the full text of an essay response in the test set

***sample_submission.csv*** - file in the required format for making predictions - note that if you are
making multiple predictions for a document, submit multiple rows 

* backbone LongFormer
* Named Entity Recognition (NER) formulation


With simple changes, we can convert this notebook into Question Answer formulation and we can try different backbones. Furthermore this notebook is one fold. It trains with 90% data and validates on 10% data. We can convert this notebook to K-fold or train with 100% data for boost in LB.

The transformer model LongFormer is explained here. It is similar to Roberta but can accept inputs as wide as 4096 tokens! In this notebook we feed the transformer with 1024 wide tokens. HuggingFace user AllenAI uploaded pretrained weights for us here

# Install and Import Libraries

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random
import os
import spacy
from spacy import displacy
from collections import Counter
import en_core_web_sm
nlp = en_core_web_sm.load()
from sklearn.datasets import fetch_20newsgroups
from sklearn.decomposition import TruncatedSVD
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import Normalizer
from sklearn import metrics
from tokenizers import *

from sklearn.cluster import KMeans, MiniBatchKMeans

import logging
from optparse import OptionParser
import sys
from time import time

from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator

from sklearn.feature_extraction.text import CountVectorizer

from gensim.models import KeyedVectors
from gensim.scripts.glove2word2vec import glove2word2vec
from sklearn.decomposition import PCA

from nltk.stem.snowball import SnowballStemmer
from itertools import cycle
plt.style.use("ggplot")
color_pal = plt.rcParams["axes.prop_cycle"].by_key()["color"]
color_cycle = cycle(plt.rcParams["axes.prop_cycle"].by_key()["color"])

Loading the data

In [ ]:
VER = 14
EPOCHS = 6
N_SPLITS = 5
MAX_LEN = 1024
TRAIN_SUBSET = None
RUN_TRAINING = False
LRS = [1e-4, 1e-4, 1e-4, 1e-4, 1e-5]
os.environ["CUDA_VISIBLE_DEVICES"]="0" #0,1,2,3 for four gpu

# VERSION FOR SAVING MODEL WEIGHTS
VER=12

# IF VARIABLE IS NONE, THEN NOTEBOOK COMPUTES TOKENS
# OTHERWISE NOTEBOOK LOADS TOKENS FROM PATH
LOAD_TOKENS_FROM = '../input/tf-longformer-v12'

# IF VARIABLE IS NONE, THEN NOTEBOOK TRAINS A NEW MODEL
# OTHERWISE IT LOADS YOUR PREVIOUSLY TRAINED MODEL
LOAD_MODEL_FROM = '../input/long-v14'

# IF FOLLOWING IS NONE, THEN NOTEBOOK 
# USES INTERNET AND DOWNLOADS HUGGINGFACE 
# CONFIG, TOKENIZER, AND MODEL
DOWNLOADED_MODEL_PATH = '../input/tf-longformer-v12'

if DOWNLOADED_MODEL_PATH is None:
    DOWNLOADED_MODEL_PATH = 'model'    
MODEL_NAME = 'allenai/longformer-base-4096'

if DOWNLOADED_MODEL_PATH == 'model':
    os.makedirs('model', exist_ok = True)
    tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
    tokenizer.save_pretrained('model')
    config = AutoConfig.from_pretrained(MODEL_NAME)
    config.save_pretrained('model')
    backbone = TFAutoModel.from_pretrained(MODEL_NAME, config=config)
    backbone.save_pretrained('model')

In [ ]:
import tensorflow as tf
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' #disable all tensorflow logging output
from transformers import *
print('TF version',tf.__version__)

In [ ]:
tf.config.optimizer.set_experimental_options({"auto_mixed_precision": True})
print('Mixed precision enabled')

# Load Data

In [ ]:
train = pd.read_csv('../input/feedback-prize-2021/train.csv')
print( train.shape )

# General Dataset Information

In [ ]:
train.info()

In [ ]:
train.head()

The column descriptions are:

*     **id** - ID code for essay response
*     **discourse_id** - ID code for discourse element
*     **discourse_start** - character position where discourse element begins in the essay response
*     **discourse_end** - character position where discourse element ends in the essay response
*     **discourse_text** - text of discourse element
*    **discourse_type**- classification of discourse element
*     **discourse_type_num**- enumerated class label of discourse element
*     **predictionstring** - the word indices of the training sample, as required for predictions


In [ ]:
assert( np.sum(train.groupby('id')['discourse_start'].diff()<=0)==0 )

In [ ]:
print('The train labels are:')
train.discourse_type.unique()

In [ ]:
IDS = train.id.unique()
print('There are',len(IDS),'train texts.')

# What Does an essay look like

lets print the first txt file AFEC37C2D43F.txt to see what an essay looks like. It's an essay about asking for advice on a certain topic or subject.


In [ ]:
!cat ../input/feedback-prize-2021/train/AFEC37C2D43F.txt

# Lets print the same example with the text colored by discourse type.

In [ ]:
def read_essay(id):
    with open(f"../input/feedback-prize-2021/train/{id}.txt") as f:
        essay = f.read()
    return essay


def text_to_color(essay, discourse_type, predictionstring):
    """
    Takes an entire essay, the discourse type and prediction string.
    Returns highlighted text for the prediction string
    """
    discourse_color_map = {
        "Lead": 1,  # 1 red
        "Position": 2,  # 2 green
        "Evidence": 3,  # 3 yellow
        "Claim": 4,  # 4 blue
        "Concluding Statement": 5,  # 5 magenta
        "Counterclaim": 6,  # 6 cyan
        "Rebuttal": 7,  # 7 white
        "None": 9,  # default
    }
    hcolor = discourse_color_map[discourse_type]
    text_index = [int(c) for c in predictionstring.split()]
    text_subset = " ".join(np.array(essay.split())[text_index])
    if discourse_type == "None":
        return f"\033[4{hcolor};30m{text_subset}\033[m"
    return f"\033[4{hcolor};30m{text_subset}\033[m"


def get_non_discourse_df(train, essay, id):
    all_pred_strings = " ".join(train.query("id == @id")["predictionstring"].values)
    all_pred_strings = [int(c) for c in all_pred_strings.split()]
    # [c for c in all_pred_strings

    non_discourse_df = pd.DataFrame(
        [c for c in range(len(essay.split())) if c not in all_pred_strings]
    )
    non_discourse_df.columns = ["predictionstring"]
    non_discourse_df["cluster"] = (
        non_discourse_df["predictionstring"].diff().fillna(1) > 1
    ).cumsum()

    non_discourse_strings = []
    for i, d in non_discourse_df.groupby("cluster"):
        pred_string = [str(x) for x in d["predictionstring"].values]
        non_discourse_strings.append(" ".join(pred_string))
    df = pd.DataFrame(non_discourse_strings).rename(columns={0: "predictionstring"})
    df["discourse_type"] = "None"
    return df


def get_colored_essay(train, id):
    essay = read_essay(id)
    all_text = ""
    train_subset = train.query("id == @id").copy()
    df = get_non_discourse_df(train, essay, id)
    train_subset = pd.concat([train_subset, df])
    train_subset["first_index"] = (
        train_subset["predictionstring"].str.split(" ").str[0].astype("int")
    )
    train_subset = train_subset.sort_values("first_index").reset_index(drop=True).copy()
    for i, d in train_subset.iterrows():
        colored_text = text_to_color(essay, d.discourse_type, d.predictionstring)
        all_text += " " + colored_text
    return all_text[1:]


all_text = get_colored_essay(train, "AFEC37C2D43F")
print(all_text)

In [ ]:
train_f = os.listdir('../input/feedback-prize-2021/train')

for file in range(len(train_f)):
    train_f[file] = str('../input/feedback-prize-2021/train') + "/" +  str(train_f[file])

# Displaying Labels

In [ ]:
j = 40
ents = []
for i, row in train[train['id'] == train_f[j][35:-4]].iterrows():
    ents.append({
                    'start': int(row['discourse_start']), 
                     'end': int(row['discourse_end']), 
                     'label': row['discourse_type']
                })
with open(train_f[j], 'r') as file: data = file.read()

doc2 = {
    "text": data,
    "ents": ents,
}
cols = {'Lead': '#dad1f6','Position': '#f9d5de','Claim': '#adcfad','Evidence': '#fbbf9a','Counterclaim': '#bdf2fa','Concluding Statement': '#eea69e','Rebuttal': '#d1f8f4'}
options = {"ents": train.discourse_type.unique().tolist(), "colors": cols}
displacy.render(doc2, style="ent", options=options, manual=True, jupyter=True);

# What Type of Annotations are Most Common

In [ ]:
ax = (
    train["discourse_type"]
    .value_counts(ascending=True)
    .plot(kind="barh", figsize=(10, 5))
)
ax.set_title("Discourse Label Frequency (in train)", fontsize=16)
ax.bar_label(ax.containers[0], label_type="edge")
plt.show()

# Exploratory Data Analysis

In [ ]:
train['full_text'] = train['discourse_text'].groupby(train['id']).transform(lambda x: ' '.join(x)) # obviously we will have duplicates

**Let's see for example the first essay:**

In [ ]:
train.full_text.iloc[0]

# Distribution of top n-grams for full-text essays

In [ ]:
def get_top_n_words(corpus, n=None, remove_stop_words=False, n_words=1): # if n_words=1 -> unigrams, if n_words=2 -> bigrams..
    if remove_stop_words:
        vec = CountVectorizer(stop_words = 'english', ngram_range=(n_words, n_words)).fit(corpus)
    else:
        vec = CountVectorizer(ngram_range=(n_words, n_words)).fit(corpus)
    bag_of_words = vec.transform(corpus)
    sum_words = bag_of_words.sum(axis=0) 
    words_freq = [(word, sum_words[0, idx]) for word, idx in vec.vocabulary_.items()]
    words_freq =sorted(words_freq, key = lambda x: x[1], reverse=True)
    return words_freq[:n]

**Unigrams**

In [ ]:
common_words = get_top_n_words(train['full_text'].drop_duplicates(), 20, remove_stop_words=True, n_words=1)
for word, freq in common_words:
    print(word, freq)

**Bigrams**

In [ ]:
common_words = get_top_n_words(train['full_text'].drop_duplicates(), 20, remove_stop_words=True, n_words=2)
for word, freq in common_words:
    print(word, freq)

**Trigrams**

In [ ]:
common_words = get_top_n_words(train['full_text'].drop_duplicates(), 20, remove_stop_words=True, n_words=3)
for word, freq in common_words:
    print(word, freq)

# Distribution of top n-grams for each discourse type

**Unigrams**

In [ ]:
text_Lead = train[train.discourse_type == 'Lead'].discourse_text.values
text_Position = train[train.discourse_type == 'Position'].discourse_text.values
text_Evidence = train[train.discourse_type == 'Evidence'].discourse_text.values
text_Claim = train[train.discourse_type == 'Claim'].discourse_text.values
text_Concluding_Statement = train[train.discourse_type == 'Concluding Statement'].discourse_text.values
text_Counterclaim = train[train.discourse_type == 'Counterclaim'].discourse_text.values
text_Rebuttal = train[train.discourse_type == 'Rebuttal'].discourse_text.values

common_words_Lead = get_top_n_words(text_Lead, 20, remove_stop_words=True, n_words=1)
common_words_Position = get_top_n_words(text_Position, 20, remove_stop_words=True, n_words=1)
common_words_Evidence = get_top_n_words(text_Evidence, 20, remove_stop_words=True, n_words=1)
common_words_Claim = get_top_n_words(text_Claim, 20, remove_stop_words=True, n_words=1)
common_words_Concluding_Statement = get_top_n_words(text_Concluding_Statement, 20, remove_stop_words=True, n_words=1)
common_words_Counterclaim = get_top_n_words(text_Counterclaim, 20, remove_stop_words=True, n_words=1)
common_words_Rebuttal = get_top_n_words(text_Rebuttal, 20, remove_stop_words=True, n_words=1)
df_tmp_Lead = pd.DataFrame(common_words_Lead, columns = ['text' , 'count'])
df_tmp_Position = pd.DataFrame(common_words_Position, columns = ['text' , 'count'])
df_tmp_Evidence = pd.DataFrame(common_words_Evidence, columns = ['text' , 'count'])
df_tmp_Claim = pd.DataFrame(common_words_Claim, columns = ['text' , 'count'])
df_tmp_Concluding_Statement = pd.DataFrame(common_words_Concluding_Statement, columns = ['text' , 'count'])
df_tmp_Counterclaim = pd.DataFrame(common_words_Counterclaim, columns = ['text' , 'count'])
df_tmp_Rebuttal = pd.DataFrame(common_words_Rebuttal, columns = ['text' , 'count'])

fig = plt.figure(figsize=(15,6))

ax1 = fig.add_subplot(121)
ax1 = df_tmp_Lead.groupby('text').sum()['count'].sort_values(ascending=False).plot(
    kind='bar', color = "#120f7a")
ax1.set_title('Lead Unigram Distribution')
ax1.set_xlabel("Unigrams")
ax1.set_ylabel("Frequency")

ax2 = fig.add_subplot(122)
ax2 = df_tmp_Position.groupby('text').sum()['count'].sort_values(ascending=False).plot(
   kind='bar', color = "#120f7a")
ax2.set_title('Position Unigram Distribution')
ax2.set_xlabel("Unigrams")
ax2.set_ylabel("Frequency")

fig = plt.figure(figsize=(15,6))

ax1 = fig.add_subplot(121)
ax1 = df_tmp_Evidence.groupby('text').sum()['count'].sort_values(ascending=False).plot(
    kind='bar', color = "#120f7a")
ax1.set_title('Evidence Unigram Distribution')
ax1.set_xlabel("Unigrams")
ax1.set_ylabel("Frequency")

ax2 = fig.add_subplot(122)
ax2 = df_tmp_Claim.groupby('text').sum()['count'].sort_values(ascending=False).plot(
   kind='bar', color = "#120f7a")
ax2.set_title('Claim Unigram Distribution')
ax2.set_xlabel("Unigrams")
ax2.set_ylabel("Frequency")

fig = plt.figure(figsize=(15,6))

ax1 = fig.add_subplot(121)
ax1 = df_tmp_Concluding_Statement.groupby('text').sum()['count'].sort_values(ascending=False).plot(
    kind='bar', color = "#120f7a")
ax1.set_title('Concluding Statement Unigram Distribution')
ax1.set_xlabel("Unigrams")
ax1.set_ylabel("Frequency")

ax2 = fig.add_subplot(122)
ax2 = df_tmp_Counterclaim.groupby('text').sum()['count'].sort_values(ascending=False).plot(
   kind='bar', color = "#120f7a")
ax2.set_title('Counterclaim Unigram Distribution')
ax2.set_xlabel("Unigrams")
ax2.set_ylabel("Frequency")

fig = plt.figure(figsize=(15,6))

ax1 = fig.add_subplot(121)
ax1 = df_tmp_Rebuttal.groupby('text').sum()['count'].sort_values(ascending=False).plot(
    kind='bar', color = "#120f7a")
ax1.set_title('Rebuttal Unigram Distribution')
ax1.set_xlabel("Unigrams")
ax1.set_ylabel("Frequency")

plt.show()



**Bigrams**

In [ ]:
text_Lead = train[train.discourse_type == 'Lead'].discourse_text.values
text_Position = train[train.discourse_type == 'Position'].discourse_text.values
text_Evidence = train[train.discourse_type == 'Evidence'].discourse_text.values
text_Claim = train[train.discourse_type == 'Claim'].discourse_text.values
text_Concluding_Statement = train[train.discourse_type == 'Concluding Statement'].discourse_text.values
text_Counterclaim = train[train.discourse_type == 'Counterclaim'].discourse_text.values
text_Rebuttal = train[train.discourse_type == 'Rebuttal'].discourse_text.values

common_words_Lead = get_top_n_words(text_Lead, 20, remove_stop_words=True, n_words=2)
common_words_Position = get_top_n_words(text_Position, 20, remove_stop_words=True, n_words=2)
common_words_Evidence = get_top_n_words(text_Evidence, 20, remove_stop_words=True, n_words=2)
common_words_Claim = get_top_n_words(text_Claim, 20, remove_stop_words=True, n_words=2)
common_words_Concluding_Statement = get_top_n_words(text_Concluding_Statement, 20, remove_stop_words=True, n_words=2)
common_words_Counterclaim = get_top_n_words(text_Counterclaim, 20, remove_stop_words=True, n_words=2)
common_words_Rebuttal = get_top_n_words(text_Rebuttal, 20, remove_stop_words=True, n_words=2)
df_tmp_Lead = pd.DataFrame(common_words_Lead, columns = ['text' , 'count'])
df_tmp_Position = pd.DataFrame(common_words_Position, columns = ['text' , 'count'])
df_tmp_Evidence = pd.DataFrame(common_words_Evidence, columns = ['text' , 'count'])
df_tmp_Claim = pd.DataFrame(common_words_Claim, columns = ['text' , 'count'])
df_tmp_Concluding_Statement = pd.DataFrame(common_words_Concluding_Statement, columns = ['text' , 'count'])
df_tmp_Counterclaim = pd.DataFrame(common_words_Counterclaim, columns = ['text' , 'count'])
df_tmp_Rebuttal = pd.DataFrame(common_words_Rebuttal, columns = ['text' , 'count'])

fig = plt.figure(figsize=(15,6))

ax1 = fig.add_subplot(121)
ax1 = df_tmp_Lead.groupby('text').sum()['count'].sort_values(ascending=False).plot(
    kind='bar', color = "#120f7a")
ax1.set_title('Lead Bigram Distribution')
ax1.set_xlabel("Bigrams")
ax1.set_ylabel("Frequency")

ax2 = fig.add_subplot(122)
ax2 = df_tmp_Position.groupby('text').sum()['count'].sort_values(ascending=False).plot(
   kind='bar', color = "#120f7a")
ax2.set_title('Position Bigram Distribution')
ax2.set_xlabel("Bigrams")
ax2.set_ylabel("Frequency")

fig = plt.figure(figsize=(15,6))

ax1 = fig.add_subplot(121)
ax1 = df_tmp_Evidence.groupby('text').sum()['count'].sort_values(ascending=False).plot(
    kind='bar', color = "#120f7a")
ax1.set_title('Evidence Bigram Distribution')
ax1.set_xlabel("Bigrams")
ax1.set_ylabel("Frequency")

ax2 = fig.add_subplot(122)
ax2 = df_tmp_Claim.groupby('text').sum()['count'].sort_values(ascending=False).plot(
   kind='bar', color = "#120f7a")
ax2.set_title('Claim Bigram Distribution')
ax2.set_xlabel("Bigrams")
ax2.set_ylabel("Frequency")

fig = plt.figure(figsize=(15,6))

ax1 = fig.add_subplot(121)
ax1 = df_tmp_Concluding_Statement.groupby('text').sum()['count'].sort_values(ascending=False).plot(
    kind='bar', color = "#120f7a")
ax1.set_title('Concluding Statement Bigram Distribution')
ax1.set_xlabel("Bigrams")
ax1.set_ylabel("Frequency")

ax2 = fig.add_subplot(122)
ax2 = df_tmp_Counterclaim.groupby('text').sum()['count'].sort_values(ascending=False).plot(
   kind='bar', color = "#120f7a")
ax2.set_title('Counterclaim Bigram Distribution')
ax2.set_xlabel("Bigrams")
ax2.set_ylabel("Frequency")

fig = plt.figure(figsize=(15,6))

ax1 = fig.add_subplot(121)
ax1 = df_tmp_Rebuttal.groupby('text').sum()['count'].sort_values(ascending=False).plot(
    kind='bar', color = "#120f7a")
ax1.set_title('Rebuttal Bigram Distribution')
ax1.set_xlabel("Bigrams")
ax1.set_ylabel("Frequency")

plt.show()

**Trigrams**

In [ ]:
text_Lead = train[train.discourse_type == 'Lead'].discourse_text.values
text_Position = train[train.discourse_type == 'Position'].discourse_text.values
text_Evidence = train[train.discourse_type == 'Evidence'].discourse_text.values
text_Claim = train[train.discourse_type == 'Claim'].discourse_text.values
text_Concluding_Statement = train[train.discourse_type == 'Concluding Statement'].discourse_text.values
text_Counterclaim = train[train.discourse_type == 'Counterclaim'].discourse_text.values
text_Rebuttal = train[train.discourse_type == 'Rebuttal'].discourse_text.values

common_words_Lead = get_top_n_words(text_Lead, 20, remove_stop_words=True, n_words=3)
common_words_Position = get_top_n_words(text_Position, 20, remove_stop_words=True, n_words=3)
common_words_Evidence = get_top_n_words(text_Evidence, 20, remove_stop_words=True, n_words=3)
common_words_Claim = get_top_n_words(text_Claim, 20, remove_stop_words=True, n_words=3)
common_words_Concluding_Statement = get_top_n_words(text_Concluding_Statement, 20, remove_stop_words=True, n_words=3)
common_words_Counterclaim = get_top_n_words(text_Counterclaim, 20, remove_stop_words=True, n_words=3)
common_words_Rebuttal = get_top_n_words(text_Rebuttal, 20, remove_stop_words=True, n_words=3)
df_tmp_Lead = pd.DataFrame(common_words_Lead, columns = ['text' , 'count'])
df_tmp_Position = pd.DataFrame(common_words_Position, columns = ['text' , 'count'])
df_tmp_Evidence = pd.DataFrame(common_words_Evidence, columns = ['text' , 'count'])
df_tmp_Claim = pd.DataFrame(common_words_Claim, columns = ['text' , 'count'])
df_tmp_Concluding_Statement = pd.DataFrame(common_words_Concluding_Statement, columns = ['text' , 'count'])
df_tmp_Counterclaim = pd.DataFrame(common_words_Counterclaim, columns = ['text' , 'count'])
df_tmp_Rebuttal = pd.DataFrame(common_words_Rebuttal, columns = ['text' , 'count'])

fig = plt.figure(figsize=(15,6))

ax1 = fig.add_subplot(121)
ax1 = df_tmp_Lead.groupby('text').sum()['count'].sort_values(ascending=False).plot(
    kind='bar', color = "#120f7a")
ax1.set_title('Lead Trigram Distribution')
ax1.set_xlabel("Trigrams")
ax1.set_ylabel("Frequency")

ax2 = fig.add_subplot(122)
ax2 = df_tmp_Position.groupby('text').sum()['count'].sort_values(ascending=False).plot(
   kind='bar', color = "#120f7a")
ax2.set_title('Position Trigram Distribution')
ax2.set_xlabel("Trigrams")
ax2.set_ylabel("Frequency")

fig = plt.figure(figsize=(15,6))

ax1 = fig.add_subplot(121)
ax1 = df_tmp_Evidence.groupby('text').sum()['count'].sort_values(ascending=False).plot(
    kind='bar', color = "#120f7a")
ax1.set_title('Evidence Trigram Distribution')
ax1.set_xlabel("Trigrams")
ax1.set_ylabel("Frequency")

ax2 = fig.add_subplot(122)
ax2 = df_tmp_Claim.groupby('text').sum()['count'].sort_values(ascending=False).plot(
   kind='bar', color = "#120f7a")
ax2.set_title('Claim Trigram Distribution')
ax2.set_xlabel("Trigrams")
ax2.set_ylabel("Frequency")

fig = plt.figure(figsize=(15,6))

ax1 = fig.add_subplot(121)
ax1 = df_tmp_Concluding_Statement.groupby('text').sum()['count'].sort_values(ascending=False).plot(
    kind='bar', color = "#120f7a")
ax1.set_title('Concluding Statement Trigram Distribution')
ax1.set_xlabel("Trigrams")
ax1.set_ylabel("Frequency")

ax2 = fig.add_subplot(122)
ax2 = df_tmp_Counterclaim.groupby('text').sum()['count'].sort_values(ascending=False).plot(
   kind='bar', color = "#120f7a")
ax2.set_title('Counterclaim Trigram Distribution')
ax2.set_xlabel("Trigrams")
ax2.set_ylabel("Frequency")

fig = plt.figure(figsize=(15,6))

ax1 = fig.add_subplot(121)
ax1 = df_tmp_Rebuttal.groupby('text').sum()['count'].sort_values(ascending=False).plot(
    kind='bar', color = "#120f7a")
ax1.set_title('Rebuttal Trigram Distribution')
ax1.set_xlabel("Trigrams")
ax1.set_ylabel("Frequency")

plt.show()

In [ ]:
ax = (
    train.groupby("discourse_type")[["discourse_start", "discourse_end"]]
    .mean()
    .sort_values("discourse_start")
    .plot(
        kind="barh",
        figsize=(10, 5),
    )
)
ax.set_title("Average Discourse Label Start and End", fontsize=16)
plt.show()

# Wordclouds by Discourse Type

In [ ]:
plt.style.use('default')

fig, axs = plt.subplots(7, 1, figsize=(20, 25))

plt_idx = 0

for discourse_type, d in train.groupby("discourse_type"):
    discourse_text = " ".join(d["discourse_text"].values.tolist())
    wordcloud = WordCloud(
        max_font_size=200,
        max_words=200,
        width=1200,
        height=800,
        background_color="white",
    ).generate(discourse_text)
    axs = axs.flatten()
    axs[plt_idx].imshow(wordcloud, interpolation="bilinear")
    axs[plt_idx].set_title(discourse_type, fontsize=18)
    axs[plt_idx].axis("off")
    plt_idx += 1
plt.tight_layout()
plt.show()

# Tokenize Train
The following code converts train dataset into a NER token array that we can use to train a NER transformer. I have made it very clear which targets belong to which class. This allows us to very easily convert this code to Question Answer formulation if we want. Just change the 14 NER arrays to be 14 arrays of start position and end position for each of the 7 classes. (You will need to think creatively what to do if a single text has multiple of one class).

In [ ]:
MAX_LEN = 1024
# THE TOKENS AND ATTENTION ARRAYS
tokenizer = AutoTokenizer.from_pretrained(DOWNLOADED_MODEL_PATH)
train_tokens = np.zeros((len(IDS),MAX_LEN), dtype='int32')
train_attention = np.zeros((len(IDS),MAX_LEN), dtype='int32')

# THE 14 CLASSES FOR NER
lead_b = np.zeros((len(IDS),MAX_LEN))
lead_i = np.zeros((len(IDS),MAX_LEN))

position_b = np.zeros((len(IDS),MAX_LEN))
position_i = np.zeros((len(IDS),MAX_LEN))

evidence_b = np.zeros((len(IDS),MAX_LEN))
evidence_i = np.zeros((len(IDS),MAX_LEN))

claim_b = np.zeros((len(IDS),MAX_LEN))
claim_i = np.zeros((len(IDS),MAX_LEN))

conclusion_b = np.zeros((len(IDS),MAX_LEN))
conclusion_i = np.zeros((len(IDS),MAX_LEN))

counterclaim_b = np.zeros((len(IDS),MAX_LEN))
counterclaim_i = np.zeros((len(IDS),MAX_LEN))

rebuttal_b = np.zeros((len(IDS),MAX_LEN))
rebuttal_i = np.zeros((len(IDS),MAX_LEN))

train_lens = []
targets_b = [lead_b, position_b, evidence_b, claim_b, conclusion_b, counterclaim_b, rebuttal_b]
targets_i = [lead_i, position_i, evidence_i, claim_i, conclusion_i, counterclaim_i, rebuttal_i]
target_map = {'Lead':0, 'Position':1, 'Evidence':2, 'Claim':3, 'Concluding Statement':4, 'Counterclaim':5, 'Rebuttal':6}

for id_num in range(len(IDS)):
    if LOAD_TOKENS_FROM: break
    if id_num % 100 == 0: print(id_num,', ',end='')
    n = IDS[id_num]
    name = f'../input/feedback-prize-2021/train/{n}.txt'
    txt = open(name, 'r').read()
    train_lens.append( len(txt.split()))
    tokens = tokenizer.encode_plus(txt, max_length=MAX_LEN, padding='max_length', truncation=True, return_offsets_mapping=True)
    train_tokens[id_num,] = tokens['input_ids']
    train_attention[id_num,] = tokens['attention_mask']
    offsets = tokens['offset_mapping']
    offset_index = 0
    df = train.loc[train.id==n]
    for index,row in df.iterrows():
        a = row.discourse_start
        b = row.discourse_end
        if offset_index>len(offsets)-1:
            break
        c = offsets[offset_index][0]
        d = offsets[offset_index][1]
        beginning = True
        while b>c:
            if (c>=a)&(b>=d):
                k = target_map[row.discourse_type]
                if beginning:
                    targets_b[k][id_num][offset_index] = 1
                    beginning = False
                else:
                    targets_i[k][id_num][offset_index] = 1
            offset_index += 1
            if offset_index>len(offsets)-1:
                break
            c = offsets[offset_index][0]
            d = offsets[offset_index][1]

if LOAD_TOKENS_FROM is None:
    targets = np.zeros((len(IDS), MAX_LEN, 15), dtype = 'int32')
    for k in range(7):
        targets[:, :, 2 * k] = targets_b[k]
        targets[:, :, 2 * k + 1] = targets_i[k]
    targets[:, :, 14] = 1 - np.max(targets, axis = -1)
if LOAD_TOKENS_FROM is None:
    np.save(f'targets_{MAX_LEN}', targets)
    np.save(f'tokens_{MAX_LEN}', train_tokens)
    np.save(f'attention_{MAX_LEN}', train_attention)
    print('Saved NER tokens')
else:
    targets = np.load(f'{LOAD_TOKENS_FROM}/targets_{MAX_LEN}.npy')
    train_tokens = np.load(f'{LOAD_TOKENS_FROM}/tokens_{MAX_LEN}.npy')
    train_attention = np.load(f'{LOAD_TOKENS_FROM}/attention_{MAX_LEN}.npy')
    print('Loaded NER tokens')

In [ ]:
if LOAD_TOKENS_FROM is None:
    plt.hist(train_lens,bins=100)
    plt.show()

From the histogram of train token counts above, we see that using a transformer width of 1024 is a good comprise of capturing most of the data's signal but not having too large a model. We could probably explore other widths between 512 and 1024 also. Or we could use widths of size 512 or smaller and use a stride which breaks a single text into multiple chunks (with possible overlap).

In [ ]:
if LOAD_TOKENS_FROM is None:
    targets = np.zeros((len(IDS),MAX_LEN,15), dtype='int32')
    for k in range(7):
        targets[:,:,2*k] = targets_b[k]
        targets[:,:,2*k+1] = targets_i[k]
    targets[:,:,14] = 1-np.max(targets,axis=-1)

In [ ]:
if LOAD_TOKENS_FROM is None:
    np.save(f'targets_{MAX_LEN}', targets)
    np.save(f'tokens_{MAX_LEN}', train_tokens)
    np.save(f'attention_{MAX_LEN}', train_attention)
    print('Saved NER tokens')
else:
    targets = np.load(f'{LOAD_TOKENS_FROM}/targets_{MAX_LEN}.npy')
    train_tokens = np.load(f'{LOAD_TOKENS_FROM}/tokens_{MAX_LEN}.npy')
    train_attention = np.load(f'{LOAD_TOKENS_FROM}/attention_{MAX_LEN}.npy')
    print('Loaded NER tokens')

# Build Model
We will use LongFormer backbone and add our own NER head using one hidden layer of size 256 and one final layer with softmax. We use 15 classes because we have a B class and I class for each of 7 labels. And we have an additional class (called O class) for tokens that do not belong to one of the 14 classes.

In [ ]:
def build_model():
    
    tokens = tf.keras.layers.Input(shape=(MAX_LEN,), name = 'tokens', dtype=tf.int32)
    attention = tf.keras.layers.Input(shape=(MAX_LEN,), name = 'attention', dtype=tf.int32)
    
    config = AutoConfig.from_pretrained(DOWNLOADED_MODEL_PATH+'/config.json') 
    backbone = TFAutoModel.from_pretrained(DOWNLOADED_MODEL_PATH+'/tf_model.h5', config=config)
    
    x = backbone(tokens, attention_mask=attention)
    x = tf.keras.layers.Dense(256, activation='relu')(x[0])
    x = tf.keras.layers.Dense(15, activation='softmax', dtype='float32')(x)
    
    model = tf.keras.Model(inputs=[tokens,attention], outputs=x)
    model.compile(optimizer = tf.keras.optimizers.Adam(lr = 1e-4),
                  loss = [tf.keras.losses.CategoricalCrossentropy()],
                  metrics = [tf.keras.metrics.CategoricalAccuracy()])
    
    return model

In [ ]:
strategy = tf.distribute.MirroredStrategy()


In [ ]:
srategy = tf.distribute.MirroredStrategy(["GPU:0","GPU:1"])
def value_fn(ctx):
    return tf.consstant(1.)

In [ ]:
with strategy.scope():
    model = build_model()

# Train or Load Model
If you provide a path in variable LOAD_MODEL_FROM above, then it will load your previously trained model. Otherwise it will train now.

We train 5 epochs of batch size 32 using learning rate 1e-4 for the first four and 1e-5 for the last epoch. I trained my model offline.

In [ ]:
# TRAIN VALID SPLIT 90% 10%
np.random.seed(123)
train_idx = np.random.choice(np.arange(len(IDS)),int(0.9*len(IDS)),replace=False)
valid_idx = np.setdiff1d(np.arange(len(IDS)),train_idx)
np.random.seed(None)
print('Train size',len(train_idx),', Valid size',len(valid_idx))

In [ ]:
# LEARNING RATE SCHEDULE AND MODEL CHECKPOINT
EPOCHS = 7
LRS = [1e-4, 1e-4, 1e-4, 1e-5, 1e-5]
def lrfn(epoch):
    return LRS[epoch]
lr_callback = tf.keras.callbacks.LearningRateScheduler(lrfn, verbose = True)

In [ ]:
if LOAD_MODEL_FROM:
    model.load_weights(f'{LOAD_MODEL_FROM}/long_v14.h5')
    
# OR TRAIN MODEL
else:
    model.fit(x = [train_tokens[train_idx,], train_attention[train_idx,]],
          y = targets[train_idx,],
          validation_data = ([train_tokens[valid_idx,], train_attention[valid_idx,]],
                             targets[valid_idx,]),
          callbacks = [lr_callback],
          epochs = EPOCHS,
          batch_size = 32,
          verbose = 2)

    # SAVE MODEL WEIGHTS
    model.save_weights(f'long_v{VER}.h5')

# Validate Model - Infer Out of fold (OOF)
We will now make predictions on the validation texts. Our model makes label predictions for each token, we need to convert this into a list of word indices for each label. Note that the tokens and words are not the same. A single word may be broken into multiple tokens. Therefore we need to first create a map to change token indices to word indices.

In [ ]:
p = model.predict([train_tokens[valid_idx,], train_attention[valid_idx,]], 
                  batch_size=16, verbose=2)
print('OOF predictions shape:',p.shape)
oof_preds = np.argmax(p,axis=-1)

In [ ]:
target_map_rev = {0:'Lead', 1:'Position', 2:'Evidence', 3:'Claim', 4:'Concluding Statement',
             5:'Counterclaim', 6:'Rebuttal', 7:'blank'}

In [ ]:
def get_preds(dataset='train', verbose=True, text_ids=IDS[valid_idx], preds=oof_preds):
    all_predictions = []

    for id_num in range(len(preds)):
    
        # GET ID
        if (id_num%100==0)&(verbose): 
            print(id_num,', ',end='')
        n = text_ids[id_num]
    
        # GET TOKEN POSITIONS IN CHARS
        name = f'../input/feedback-prize-2021/{dataset}/{n}.txt'
        txt = open(name, 'r').read()
        
        '''txt = txt.replace(' .', '. ')
        txt = txt.replace(' ,', ', ')'''
        
        tokens = tokenizer.encode_plus(txt, max_length=MAX_LEN, padding='max_length',
                                   truncation=True, return_offsets_mapping=True)
        off = tokens['offset_mapping']
    
        # GET WORD POSITIONS IN CHARS
        w = []
        blank = True
        for i in range(len(txt)):
            if (txt[i]!=' ')&(txt[i]!='\n')&(txt[i]!='\xa0')&(txt[i]!='\x85')&(blank==True):
                w.append(i)
                blank=False
            elif (txt[i]==' ')|(txt[i]=='\n')|(txt[i]=='\xa0')|(txt[i]=='\x85'):
                blank=True
        w.append(1e6)
            
        # MAPPING FROM TOKENS TO WORDS
        word_map = -1 * np.ones(MAX_LEN,dtype='int32')
        w_i = 0
        for i in range(len(off)):
            if off[i][1]==0: continue #ignore first blank
            while off[i][0]>=w[w_i+1]: w_i += 1
            word_map[i] = int(w_i)
        
        # CONVERT TOKEN PREDICTIONS INTO WORD LABELS
        # KEY:
        # 0: LEAD_B, 1: LEAD_I
        # 2: POSITION_B, 3: POSITION_I
        # 4: EVIDENCE_B, 5: EVIDENCE_I
        # 6: CLAIM_B, 7: CLAIM_I
        # 8: CONCLUSION_B, 9: CONCLUSION_I
        # 10: COUNTERCLAIM_B, 11: COUNTERCLAIM_I
        # 12: REBUTTAL_B, 13: REBUTTAL_I
        # 14: NOTHING i.e. O
        pred = preds[id_num,]/2.0
    
        i = 0
        while i<MAX_LEN:
            prediction = []
            start = pred[i]
            if start in [0,1,2,3,4,5,6,7]:
                prediction.append(word_map[i])
                i += 1
                if i>=MAX_LEN: break
                while pred[i]==start+0.5:
                    if not word_map[i] in prediction:
                        prediction.append(word_map[i])
                    i += 1
                    if i>=MAX_LEN: break
            else:
                i += 1
            prediction = [x for x in prediction if x!=-1]
            if len(prediction)>=4:
                all_predictions.append( (n, target_map_rev[int(start)], 
                                ' '.join([str(x) for x in prediction]) ) )
                
    # MAKE DATAFRAME
    df = pd.DataFrame(all_predictions)
    df.columns = ['id','class','predictionstring']
    
    return df

In [ ]:
oof = get_preds( dataset='train', verbose=True, text_ids=IDS[valid_idx] )
oof.head()

In [ ]:
print('The following classes are present in oof preds:')
oof['class'].unique()

# Compute Validation Metric

In [ ]:
def calc_overlap(row):
    """
    Calculates the overlap between prediction and
    ground truth and overlap percentages used for determining
    true positives.
    """
    set_pred = set(row.predictionstring_pred.split(' '))
    set_gt = set(row.predictionstring_gt.split(' '))
    # Length of each and intersection
    len_gt = len(set_gt)
    len_pred = len(set_pred)
    inter = len(set_gt.intersection(set_pred))
    overlap_1 = inter / len_gt
    overlap_2 = inter/ len_pred
    return [overlap_1, overlap_2]


def score_feedback_comp(pred_df, gt_df):
    """
    A function that scores for the kaggle
        Student Writing Competition
        
    Uses the steps in the evaluation page here:
        https://www.kaggle.com/c/feedback-prize-2021/overview/evaluation
    """
    gt_df = gt_df[['id','discourse_type','predictionstring']] \
        .reset_index(drop=True).copy()
    pred_df = pred_df[['id','class','predictionstring']] \
        .reset_index(drop=True).copy()
    pred_df['pred_id'] = pred_df.index
    gt_df['gt_id'] = gt_df.index
    # Step 1. all ground truths and predictions for a given class are compared.
    joined = pred_df.merge(gt_df,
                           left_on=['id','class'],
                           right_on=['id','discourse_type'],
                           how='outer',
                           suffixes=('_pred','_gt')
                          )
    joined['predictionstring_gt'] = joined['predictionstring_gt'].fillna(' ')
    joined['predictionstring_pred'] = joined['predictionstring_pred'].fillna(' ')

    joined['overlaps'] = joined.apply(calc_overlap, axis=1)


    joined['overlap1'] = joined['overlaps'].apply(lambda x: eval(str(x))[0])
    joined['overlap2'] = joined['overlaps'].apply(lambda x: eval(str(x))[1])


    joined['potential_TP'] = (joined['overlap1'] >= 0.5) & (joined['overlap2'] >= 0.5)
    joined['max_overlap'] = joined[['overlap1','overlap2']].max(axis=1)
    tp_pred_ids = joined.query('potential_TP') \
        .sort_values('max_overlap', ascending=False) \
        .groupby(['id','predictionstring_gt']).first()['pred_id'].values

    # 3. Any unmatched ground truths are false negatives
    # and any unmatched predictions are false positives.
    fp_pred_ids = [p for p in joined['pred_id'].unique() if p not in tp_pred_ids]

    matched_gt_ids = joined.query('potential_TP')['gt_id'].unique()
    unmatched_gt_ids = [c for c in joined['gt_id'].unique() if c not in matched_gt_ids]

    # Get numbers of each type
    TP = len(tp_pred_ids)
    FP = len(fp_pred_ids)
    FN = len(unmatched_gt_ids)
    #calc microf1
    my_f1_score = TP / (TP + 0.5*(FP+FN))
    return my_f1_score

In [ ]:
valid = train.loc[train['id'].isin(IDS[valid_idx])]

In [ ]:
f1s = []
CLASSES = oof['class'].unique()
for c in CLASSES:
    pred_df = oof.loc[oof['class']==c].copy()
    gt_df = valid.loc[valid['discourse_type']==c].copy()
    f1 = score_feedback_comp(pred_df, gt_df)
    print(c,f1)
    f1s.append(f1)
print()
print('Overall',np.mean(f1s))

In [ ]:
files = os.listdir('../input/feedback-prize-2021/test')
TEST_IDS = [f.replace('.txt','') for f in files if 'txt' in f]
print('There are',len(TEST_IDS),'test texts.')

In [ ]:
test_tokens = np.zeros((len(TEST_IDS),MAX_LEN), dtype='int32')
test_attention = np.zeros((len(TEST_IDS),MAX_LEN), dtype='int32')

for id_num in range(len(TEST_IDS)):
        
    # READ TRAIN TEXT, TOKENIZE, AND SAVE IN TOKEN ARRAYS    
    n = TEST_IDS[id_num]
    name = f'../input/feedback-prize-2021/test/{n}.txt'
    txt = open(name, 'r').read()
    
    tokens = tokenizer.encode_plus(txt, max_length=MAX_LEN, padding='max_length',
                                   truncation=True, return_offsets_mapping=True)
    test_tokens[id_num,] = tokens['input_ids']
    test_attention[id_num,] = tokens['attention_mask']

In [ ]:
# INFER TEST TEXTS
p = model.predict([test_tokens, test_attention], 
                  batch_size=16, verbose=2)
print('Test predictions shape:',p.shape)
test_preds = np.argmax(p,axis=-1)

In [ ]:
# GET TEST PREDICIONS
sub = get_preds( dataset='test', verbose=False, text_ids=TEST_IDS, preds=test_preds )
sub.head()

In [ ]:
files = os.listdir('../input/feedback-prize-2021/test')
TEST_IDS = [f.replace('.txt','') for f in files if 'txt' in f]
for id_num in range(len(TEST_IDS)):
        
    n = TEST_IDS[id_num]
    name = f'../input/feedback-prize-2021/test/{n}.txt'
    txt = open(name, 'r').read()
    
    txt = txt.replace('?','.')
    phrases = txt.split('.')[0:-1]
    
    phrase_start = [0]
    for phrase in phrases:
        phrase_len = len(phrase.split())
        phrase_start.append(phrase_start[-1]+phrase_len)
        
    '''print(phrase_start)
    print(' '.join(txt.split()[65:84]))'''
    predstrings = sub.loc[sub.id==TEST_IDS[id_num]]['predictionstring']
    
    corr_predstrings=[]
    for i, predstring in enumerate(predstrings):
        predstart = int(predstring.split()[0])
        predend = int(predstring.split()[-1])
        
        for j in range(len(phrase_start)-1):
            if (predstart > phrase_start[j]) & (predstart < phrase_start[j+1]):
                predstart = phrase_start[j]
            if (predend > phrase_start[j]) & (predend < phrase_start[j+1]):
                predend = phrase_start[j+1]
            
        predstring = ' '.join([str(val) for val in range(predstart,predend)])
        corr_predstrings.append(predstring)
    
    sub.loc[sub.id==TEST_IDS[id_num], 'predictionstring'] = corr_predstrings
    
    
        
sub.head()

# Write Submission CSV
# 

In [ ]:
sub.to_csv('submission.csv',index=False)